In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
from scipy.optimize import curve_fit


## Creating Bak-Sneppen model

Steps:
Inititalize percolation grid
Start time loop
For every time iteration:
    Calculate probabilities that neighbours change

In [ ]:
class SimulationGrid:
    def __init__(self, size, lambda_rate = 0.9, low=0.0, high=1.0):
        self.size = size
        self.threshold = []
        self.low = low
        self.high = high
        self.fitness_max = -1
        self.minima = []
        self.history = []
        self.num_iterations = 0
        self.avalanche_length = []
        self.current = self.initialize_grid()
        self.lambda_rate = lambda_rate  # Adjust as needed
        # Schedule the first information event
        self.time_for_next_info = np.random.exponential(scale=1/self.lambda_rate)
        
    def restart(self):
        self.current = self.initialize_grid()
        self.history = []
        self.threshold_fitness = []
        self.fitness_max = -1

    def initialize_grid(self):
        return np.array([[np.random.uniform(self.low, self.high) for _ in range(self.size)] for _ in range(self.size)])

    def visualize(self):
        cmap = 'viridis'
        plt.imshow(self.current, cmap=cmap, interpolation='nearest')
        plt.title('2D Grid Initialized with Uniform Distribution')
        plt.colorbar()
        plt.show()

    def draw_value(self):
#         return np.random.normal(loc=self.mean, scale=self.std, size=(self.size, self.size))
        return np.random.uniform(self.low, self.high)

    def avalanche_frequencies(self):  
        freq = {}
        for value in self.avalanche_length:
            if value in freq:
                freq[value] += 1
            else:
                freq[value] = 1
                
        lst = np.array([[key,freq[key]] for key in freq])
        sorted_indices = np.argsort(lst[:, 0])
        # Use the sorted indices to sort the array
        sorted_frequencies = lst[sorted_indices]
        
        return sorted_frequencies
    
    def introduce_new_information(self, current_time, num_clusters, fitness_value, lambda_rate):
        """
        Introduces a cluster of new information based on an exponential distribution to 
        determine the timing of events.
        
        :param current_time: The current time step in the simulation.
        :param fitness_value: The fitness value to assign to the new information clusters.
        """
        if current_time >= self.time_for_next_info:
            # Determine the number of clusters to introduce (can be based on Poisson or another approach)
            for _ in range(num_clusters):
                # Select a random site for the new information
                site = np.random.randint(0, self.size)
                # Update self.current instead of self.grid
                i, j = np.unravel_index(site, self.current.shape)
                self.current[i, j] = fitness_value

            # Schedule the next information event
            self.time_for_next_info += np.random.exponential(scale=1/self.lambda_rate)
            
    def adjust_neighbor_fitness(self, i, j):
        """
        Adjusts the fitness of a neighbor based on its original value.
        The adjustment weight is randomly chosen from U(0,1) and the direction (+ or -)
        is determined by a Bernoulli trial.
        """
        original_value = self.current[i, j]
        weight = np.random.uniform(0, 1)  # Weight is randomly chosen from U(0,1)
        
#         change_direction = 1 if np.random.rand() < 0.5 else -1 # Direction is determined by a Bernoulli trial
        change_direction = -1
        adjustment = original_value * weight * change_direction
        new_value = original_value + adjustment

        # Ensure the new value is within the bounds -> [0,1]
        return max(min(new_value, self.high), self.low)

    def run_iterations(self, iterations, num_clusters = 0, lambda_rate = 0.9,event_fitness=0.1, save=False):
        self.num_iterations= iterations
        avalanche_size = 0
        fitness_max = 0
        for t in range(iterations):
            
            # Introduce new information if the time has come
            if num_clusters > 0:
                self.introduce_new_information(t, num_clusters, event_fitness, lambda_rate)
            
            #Choosing to save values 
            if save:
                self.history.append(self.current.copy())  # Use copy to save the state, not a reference

            # Getting min index and saving minimum value
            min_index = np.unravel_index(self.current.argmin(), self.current.shape)
            minimum = self.current[min_index[0], min_index[1]]
            self.minima.append(minimum)
            
            
            #Updating fitness threshold and saving
            avalanche_size += 1
            if fitness_max < minimum:
                    self.avalanche_length.append(avalanche_size)
                    self.threshold.append((t,minimum))
                    fitness_max = minimum
                    avalanche_size = 0 
            

            # I could make this part more efficient but too lazy for now
            neighbors = [
                (min_index[0] - 1, min_index[1]),  # Above
                (min_index[0] + 1, min_index[1]),  # Below
                (min_index[0], min_index[1] - 1),  # Left
                (min_index[0], min_index[1] + 1)   # Right
            ]

            # Keeping only 'useful' indexes
            indices = [(i, j) for i, j in neighbors if 0 <= i < self.current.shape[0] and 0 <= j < self.current.shape[1]]

#             Updating neighbors and worst value
#             for index in indices:
#                 i, j = index
#                 self.current[i, j] = self.draw_value()

#             self.current[min_index[0], min_index[1]] = self.draw_value()
            
            for i, j in indices:
#                 self.current[i, j] = self.draw_value()
                self.current[i, j] = self.adjust_neighbor_fitness(i, j)
                
            self.current[min_index[0], min_index[1]] = self.draw_value()
            
        self.threshold = np.array(self.threshold)

# An avalanche is a cascade of fitness changes below the threshold (i.e. all the blinking dots below the
# line), although this behavior also results in random fitness changes above the line. An avalanche lasts
# as long as any activity remains below the threshold, and the length of the avalanche is the number of
# mutations below the threshold.

## Running and visualizing grid

In [ ]:
grid = SimulationGrid(100)
grid.visualize()
plt.show()
#  run_iterations(self, iterations, num_clusters = 0, lambda_rate = 0.9,event_fitness=0.1, save=False)

grid.run_iterations(1000000, num_clusters= 0, save=False)
grid.visualize()
plt.show()

# plt.hist(grid.current.flatten())

plt.hist(grid.minima, bins = 100)

In [ ]:
def plot_minima_and_threshold(grid):
    s1 = np.arange(len(grid.minima))
#     plt.scatter(s1, grid.minima, s=0.1)
#     print(grid.threshold)
    s2, threshold = grid.threshold[:, 0], grid.threshold[:, 1]
#     plt.xlim(-3000, 30000)
#     plt.xlim(-3000, np.max(s2))
#     plt.scatter(s2, threshold, s= 10)
    plt.plot(s2, threshold)
    return s2, threshold

def plot_threshold(grid):
    plt.plot(grid.threshold[:, 0], grid.threshold[:, 1])

s2, threshold = plot_minima_and_threshold(grid)

### Avalanche size frequency distribution

In [ ]:
def fit_power_law(data):
    '''Fits the power law using '''
    # Define the power law function
    
    def power_law(x, m, b):
        return m*x+ b
    
    filtered_ones_log = data
    filtered_ones_log = data[data[:, 1] > 0]
    
    logx = filtered_ones_log[:,0]
    logy = filtered_ones_log[:,1]

    
    initial_guess = [2, logy[0]]
    bounds = [(-5,5), (logy[0]-0.5, logy[0]+0.5)]
    
    lower_bounds = [-np.inf,logy[0]-0.1]
    upper_bounds = [np.inf,logy[0]+0.1]
    
    params, covariance = curve_fit(power_law, logx, logy, p0=initial_guess, bounds=(lower_bounds, upper_bounds))
#     params, covariance = curve_fit(power_law, logx, logy)
    # Fit the power law function to the data

    # Get the fitted parameters
    a_fit, b_fit = params
    print(f"m is {a_fit}")
    print(f"b is {b_fit}")

    # Generate fitted curve
    x_fit = np.linspace(logx[0], logx[-1], 10)
    
    y_fit = a_fit*x_fit+(b_fit)

    return x_fit, y_fit , a_fit, b_fit


frequencies = grid.avalanche_frequencies()

log_frequencies = np.log(frequencies)
x, y, m, b  = fit_power_law(log_frequencies)
plt.scatter(log_frequencies[:,0], log_frequencies[:,1])
plt.plot(x, y)

    

## Running for different grid sizes

In [ ]:
simulations_list = []

for L in [50,100,200]:
    grid = SimulationGrid(L)
    grid.run_iterations(80000, save=False)
    simulations_list.append(grid)
    
print(type(simulations_list[0]))
# colors = ['r','g', 'b', 'o']
colors = ['blue', 'green', 'red']


for indx, grid in enumerate(simulations_list):
    print(indx)
    print(grid.size)
    name = f"L={grid.size}"
    plt.plot(grid.threshold[:, 0], grid.threshold[:, 1], color= colors[indx], label=name)

plt.title('Convergence')
plt.legend()    
    


### Plotting frequencies

In [ ]:
fig, axes = plt.subplots(1, len(simulations_list), sharey=True, sharex=True, figsize=(15, 5))

for indx, (grid, ax) in enumerate(zip(simulations_list, axes)):
    frequencies = grid.avalanche_frequencies()
    log_frequencies = np.log(frequencies)
    x, y , m ,b= fit_power_law(log_frequencies)
    name = f"m = {round(m,2)}, b = {round(b,2)}"
    ysum= np.sum(y)
    colors = ['blue', 'green', 'red']
    
    ax.scatter(log_frequencies[:, 0], log_frequencies[:, 1], color=colors[indx], label=name)
    ax.plot(x, y, color=colors[indx])

    ax.set_xlabel('log(s)')
    ax.set_ylabel('log(y)')
    ax.set_title(f'L = {grid.size}')
    ax.legend()
    # Set x-axis limits
#     ax.set_xlim(-0.2, 8)
plt.suptitle('Frequency distribution of avalanche size')

# Add legend to the last subplot
axes[-1].legend()

plt.tight_layout()
plt.show()

### Histogram distribution of fitness values

In [ ]:
# grid1, grid2, grid3 = simulations_list

# Testing convergence under different lambdas